In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

In [2]:
# Set device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Step 1: Load CIFAR-10 Dataset
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(), # Augmentation: Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4), # Augmentation: Randomly crop images with padding
    transforms.ToTensor(), # Convert image to tensor format
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize image with mean and std
])

# Load train and test datasets
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# DataLoader for batch processing
train_loader = DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=100, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Step 2: Define CNN Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Define convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        
        # Define max pooling layer
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # Define fully connected layers
        self.fc1 = nn.Linear(128 * 4 * 4, 256) # 128 channels, each 4x4 after pooling
        self.fc2 = nn.Linear(256, 10) # Output layer has 10 nodes (for 10 classes)
        
        # Define dropout to prevent overfitting
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        # Pass through convolutional layers followed by ReLU activation and pooling
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        
        # Flatten the tensor for fully connected layer
        x = x.view(-1, 128 * 4 * 4)
        
        # Pass through fully connected layers with dropout
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Instantiate the model, define loss function and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss() # Cross entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001) # Adam optimizer

In [4]:
# Step 3: Train the Model
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        # Move data to device (GPU or CPU)
        images = images.to(device)
        labels = labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        
        # Print statistics
        running_loss += loss.item()
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {running_loss / 100:.4f}')
            running_loss = 0.0


Epoch [1/10], Step [100/500], Loss: 1.9023
Epoch [1/10], Step [200/500], Loss: 1.6346
Epoch [1/10], Step [300/500], Loss: 1.5277
Epoch [1/10], Step [400/500], Loss: 1.4350
Epoch [1/10], Step [500/500], Loss: 1.3730
Epoch [2/10], Step [100/500], Loss: 1.3264
Epoch [2/10], Step [200/500], Loss: 1.3149
Epoch [2/10], Step [300/500], Loss: 1.2697
Epoch [2/10], Step [400/500], Loss: 1.2189
Epoch [2/10], Step [500/500], Loss: 1.1953
Epoch [3/10], Step [100/500], Loss: 1.1708
Epoch [3/10], Step [200/500], Loss: 1.1427
Epoch [3/10], Step [300/500], Loss: 1.1240
Epoch [3/10], Step [400/500], Loss: 1.1235
Epoch [3/10], Step [500/500], Loss: 1.1057
Epoch [4/10], Step [100/500], Loss: 1.0895
Epoch [4/10], Step [200/500], Loss: 1.0719
Epoch [4/10], Step [300/500], Loss: 1.0753
Epoch [4/10], Step [400/500], Loss: 1.0266
Epoch [4/10], Step [500/500], Loss: 1.0367
Epoch [5/10], Step [100/500], Loss: 1.0054
Epoch [5/10], Step [200/500], Loss: 1.0364
Epoch [5/10], Step [300/500], Loss: 0.9869
Epoch [5/10

In [8]:
# Step 4: Test the Model
model.eval() # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad(): # Disable gradient calculation
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Print accuracy
accuracy = 100 * correct / total
print(f'Accuracy of the model on the test images: {accuracy:.2f}%')

# Save the trained model
torch.save(model.state_dict(), 'cnn_cifar10.pth')

# v1 74.66%

Accuracy of the model on the test images: 73.95%
